In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os 

In [2]:
### Remove the unncessary characters
text_file = open('text file.txt', 'r', encoding='utf8')
lines = text_file.read()
data_into_list = data = lines.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')
data_into_list

# Remove the unnecessary spaces
data = data_into_list.split()
data = ' '.join(data)
data[:500]

'The Project Gutenberg eBook of Pride and Prejudice, by Jane AustenThis eBook is for the use of anyone anywhere in the United States andmost other parts of the world at no cost and with almost no restrictionswhatsoever. You may copy it, give it away or re-use it under the termsof the Project Gutenberg License included with this eBook or online atwww.gutenberg.org. If you are not located in the United States, youwill have to check the laws of the country where you are located beforeusing this eBoo'

##### Tokenization

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

### saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 223, 186, 912, 3, 328, 4, 1351, 30, 72, 4174, 912, 23, 21, 1]

In [4]:
len(sequence_data)

125076

In [5]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

7200


##### We will use three words to preidct the next word

In [6]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)

print(f'The length of sequneces are : {len(sequences)}')
sequences[:10]

The length of sequneces are : 125073


[[1, 223, 186, 912],
 [223, 186, 912, 3],
 [186, 912, 3, 328],
 [912, 3, 328, 4],
 [3, 328, 4, 1351],
 [328, 4, 1351, 30],
 [4, 1351, 30, 72],
 [1351, 30, 72, 4174],
 [30, 72, 4174, 912],
 [72, 4174, 912, 23]]

##### Separate input and output

In [7]:
x = []
y = []

for i in sequences:
    x.append(i[0:3])
    y.append(i[3])

x = np.array(x)
y = np.array(y)


### Label the output categorically

In [8]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

##### Defining the model

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             72000     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 7200)              7207200   
                                                                 
Total params: 20,328,200
Trainable params: 20,328,200
Non-trainable params: 0
_________________________________________________________________


##### Training the model

In [13]:
from tensorflow.keras.utils import Sequence
import numpy as np   

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(x, y, 64)

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))
model.fit(train_gen, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
1955/1955 [==============================] - ETA: 0s - loss: 5.5659
Epoch 1: loss improved from inf to 5.56594, saving model to next_words.h5
1955/1955 [==============================] - 72s 36ms/step - loss: 5.5659
Epoch 2/70
1954/1955 [============================>.] - ETA: 0s - loss: 5.3028
Epoch 2: loss improved from 5.56594 to 5.30275, saving model to next_words.h5
1955/1955 [==============================] - 70s 36ms/step - loss: 5.3027
Epoch 3/70
1954/1955 [============================>.] - ETA: 0s - loss: 5.1084
Epoch 3: loss improved from 5.30275 to 5.10840, saving model to next_words.h5
1955/1955 [==============================] - 74s 38ms/step - loss: 5.1084
Epoch 4/70
1955/1955 [==============================] - ETA: 0s - loss: 4.9379
Epoch 4: loss improved from 5.10840 to 4.93795, saving model to next_words.h5
1955/1955 [==============================] - 72s 37ms/step - loss: 4.9379
Epoch 5/70
1954/1955 [============================>.] - ETA: 0s - loss: 4.7808
E

1954/1955 [============================>.] - ETA: 0s - loss: 0.6238
Epoch 36: loss improved from 0.63420 to 0.62393, saving model to next_words.h5
1955/1955 [==============================] - 70s 36ms/step - loss: 0.6239
Epoch 37/70
1955/1955 [==============================] - ETA: 0s - loss: 0.6081
Epoch 37: loss improved from 0.62393 to 0.60806, saving model to next_words.h5
1955/1955 [==============================] - 70s 36ms/step - loss: 0.6081
Epoch 38/70
1955/1955 [==============================] - ETA: 0s - loss: 0.5966
Epoch 38: loss improved from 0.60806 to 0.59663, saving model to next_words.h5
1955/1955 [==============================] - 70s 36ms/step - loss: 0.5966
Epoch 39/70
1955/1955 [==============================] - ETA: 0s - loss: 0.5855
Epoch 39: loss improved from 0.59663 to 0.58548, saving model to next_words.h5
1955/1955 [==============================] - 70s 36ms/step - loss: 0.5855
Epoch 40/70
1954/1955 [============================>.] - ETA: 0s - loss: 0.5765


##### Predict using model

In [21]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

### Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def predict_next_words(model, tokenizer, text):
    
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    preds = np.argmax(model.predict(sequence))
    predicted_word = ""
    
    for key, value in tokenizer.word_index.items():
        if value == preds:
            predicted_word = key
            break
    
    print(predicted_word)
    return(predicted_word)


In [22]:
while(True):
    text = input("Enter your line : ")
    
    if text == "0":
        print("Execution Completed.....")
        break
        
    else :
        try:
            text = text.split(" ")
            text = text[-3:]
            print(text)
            
            predict_next_words(model, tokenizer, text)
            
        except Exception as e:
            print(f"Error occurred : {e}")
            continue

Enter your line : The Project Gutenberg
['The', 'Project', 'Gutenberg']
1/1 [==============================] - 1s 585ms/step
literary
Enter your line : The Project Gutenberg eBook of
['Gutenberg', 'eBook', 'of']
1/1 [==============================] - 0s 18ms/step
pride
Enter your line :  Lizzy is not a bit better than
['bit', 'better', 'than']
1/1 [==============================] - 0s 18ms/step
the


KeyboardInterrupt: Interrupted by user